In [5]:
from azure.cognitiveservices.language.luis.authoring import LUISAuthoringClient
from azure.cognitiveservices.language.luis.authoring.models import ApplicationCreateObject, ExampleLabelObject
from azure.cognitiveservices.language.luis.runtime import LUISRuntimeClient
from msrest.authentication import CognitiveServicesCredentials
from functools import reduce

import json, time, uuid

import keys
import requests
import re

In [2]:
authoringKey = keys.authoringKey
authoringEndpoint = keys.authoringEndpoint
predictionKey = keys.predictionKey
predictionEndpoint = keys.predictionEndpoint

In [3]:
client = LUISAuthoringClient(authoringEndpoint, CognitiveServicesCredentials(authoringKey))

## I. Creating LUIS

In [5]:
app_id = client.apps.add({
            'name': "FlyMe",
            'initial_version_id':  "0.1",
            'description': "Flight booking chatbot app",
            'culture': 'en-us',
        })

print("Created app.")

Created app.


In [4]:
app_id = "db70b5ef-3103-46a0-aa92-7cbfd39c1439"

In [8]:
luis_app = client.apps.get(app_id=app_id)

app_version = luis_app.active_version

In [7]:
entities = ['or_city', 'dst_city', 'str_date', 'end_date', 'budget']
prebuilt_entities = ['datetimeV2']


In [8]:
# Create intent into the model
intent_name = "BookFlight"

client.model.add_intent(app_id, "0.1", "BookFlight")
print("Added intent BookFlight")

Added intent BookFlight


In [9]:
# add a prebuilt entity "datetimeV2" to the model

client.model.add_prebuilt(
    app_id=app_id,
    version_id=app_version,
    prebuilt_extractor_names=["datetimeV2"]
)
print("Added prebuilt entity datetimeV2")

Added prebuilt entity datetimeV2


In [11]:
# Create entities and entity features into the model
for entity in entities:
    client.model.add_entity(app_id, "0.1", name=entity)
    print("Added entity {}".format(entity))

Added entity or_city
Added entity dst_city
Added entity str_date
Added entity end_date
Added entity budget


## II. Testing LUIS

In [12]:
with open("../data/train.json") as f:
    train_data = json.load(f)

# send the training data to the service in chunks of 100

for i in range(0, len(train_data), 100):
    j = i + 100
    if j > len(train_data):
        j = len(train_data)

        client.examples.batch(
                    app_id,
                    app_version,
                    train_data[i:j]
                )

In [13]:
async_training = client.train.train_version(app_id, app_version)
is_trained = async_training.status == "UpToDate"

trained_status = ["UpToDate", "Success"]
while not is_trained:
    time.sleep(1)
    status = client.train.get_status(app_id, app_version)
    is_trained = all(
        m.details.status in trained_status for m in status)

print("Your app is trained. You can now go to the LUIS portal and test it!")

Your app is trained. You can now go to the LUIS portal and test it!


In [15]:
# Publish the app
client.apps.update_settings(app_id, is_public=True)
publish_result = client.apps.publish(app_id, app_version, is_staging=False)

endpoint = publish_result.endpoint_url + \
            "?subscription-key=" + predictionKey + "&q="

print("The app is published.")

The app is published.


In [42]:
runtime_client = LUISRuntimeClient(predictionEndpoint, CognitiveServicesCredentials(predictionKey))

request = {
    "query" : "I want to book a flight from Strasbourg to London from 10/10/2019 to 15/10/2019 for 200 euros"
}
result = runtime_client.prediction.get_slot_prediction(app_id=app_id, slot_name="Production", prediction_request=request)

print("\nDetected intent: {}".format(
            result.prediction.top_intent
        ))
print("Detected entities:")
for entity in result.prediction.entities:
    print("\t-> Entity '{}' (type: {})".format(
        result.prediction.entities.get(entity),
        entity
    ))
print("\nComplete result object as dictionnary")
print(result.as_dict())


Detected intent: BookFlight
Detected entities:
	-> Entity '['Strasbourg']' (type: or_city)
	-> Entity '['London']' (type: dst_city)
	-> Entity '[{'type': 'daterange', 'values': [{'timex': '(2019-10-10,2019-10-15,P5D)', 'resolution': [{'start': '2019-10-10', 'end': '2019-10-15'}]}]}]' (type: datetimeV2)
	-> Entity '['200 euros']' (type: budget)

Complete result object as dictionnary
{'query': 'I want to book a flight from Strasbourg to London from 10/10/2019 to 15/10/2019 for 200 euros', 'prediction': {'top_intent': 'BookFlight', 'intents': {'BookFlight': {'score': 0.99955165}}, 'entities': {'or_city': ['Strasbourg'], 'dst_city': ['London'], 'datetimeV2': [{'type': 'daterange', 'values': [{'timex': '(2019-10-10,2019-10-15,P5D)', 'resolution': [{'start': '2019-10-10', 'end': '2019-10-15'}]}]}], 'budget': ['200 euros']}}}


Since LUIS is not so good at extracting str_date and end_date, we create a function to extract these values using the datetimeV2 prebuilt entity and search_dates from dateparser.

In [59]:
def date_extractor(luis_res):
    res_dict = luis_res.as_dict()
    dt = res_dict['prediction']['entities'].get('datetimeV2')

    if dt[0]['type'] == 'daterange':
        print("Start date: {}".format(dt[0].get('values')[0].get('resolution')[0].get('start')))
        print("End date: {}".format(dt[0].get('values')[0].get('resolution')[0].get('end')))

date_extractor(result)

Start date: 2019-10-10
End date: 2019-10-15


## III. Evaluating LUIS

In [65]:
with open("../data/test.json", encoding="utf-8") as f:
    test_data = json.load(f)

In [66]:
new_data = {
    "LabeledTestSetUtterances": [
        {
            "text": d["text"],
            "intent": d["intentName"],
            "entities": [
                {
                    "entity": label["entityName"],
                    "startPos": label["startCharIndex"],
                    "endPos": label["endCharIndex"]
                } for label in d["entityLabels"]
            ]
        } for d in test_data
    ]
}

new_json = json.dumps(new_data)

In [73]:
with open('data.json', 'w') as f:
    json.dump(new_data, f)

In [67]:
data = str(new_json)

headers = {
    'Content-Type': 'application/json',
    'Ocp-Apim-Subscription-Key': predictionKey
}

slot_name = "Production"
start_batch_endpoint = f'{predictionEndpoint}luis/v3.0-preview/apps/{app_id}/slots/{slot_name}/evaluations'

start_response = requests.post(start_batch_endpoint, data=data, headers=headers)
start_response.text

'{"operationId":"e252e9b0-f578-4282-bce0-3fd3866dfa1a_638137440000000000","status":"notstarted","createdDateTime":"03/07/2023 20:02:17","lastActionDateTime":"03/07/2023 20:02:17"}'

In [68]:
start_batch_endpoint = f'{predictionEndpoint}luis/v3.0-preview/apps/{app_id}/slots/{slot_name}/evaluations/e252e9b0-f578-4282-bce0-3fd3866dfa1a_638137440000000000/status'

start_response = requests.get(start_batch_endpoint, headers=headers)
print(start_response.text)
print(start_response)

{"operationId":"e252e9b0-f578-4282-bce0-3fd3866dfa1a_638137440000000000","status":"succeeded","createdDateTime":"03/07/2023 20:02:17","lastActionDateTime":"03/07/2023 20:02:23"}
<Response [200]>


In [69]:
start_response.json()

{'operationId': 'e252e9b0-f578-4282-bce0-3fd3866dfa1a_638137440000000000',
 'status': 'succeeded',
 'createdDateTime': '03/07/2023 20:02:17',
 'lastActionDateTime': '03/07/2023 20:02:23'}

In [70]:
start_batch_endpoint = f'{predictionEndpoint}luis/v3.0-preview/apps/{app_id}/slots/{slot_name}/evaluations/e252e9b0-f578-4282-bce0-3fd3866dfa1a_638137440000000000/result'

start_response = requests.get(start_batch_endpoint, headers=headers)

In [71]:
start_response.json().get('intentModelsStats')

[{'modelName': 'BookFlight',
  'modelType': 'Intent Classifier',
  'precision': 1.0,
  'recall': 0.99,
  'fScore': 1.0},
 {'modelName': 'None',
  'modelType': 'Intent Classifier',
  'precision': 0.0,
  'recall': 'NaN',
  'fScore': 'NaN'}]

In [72]:
start_response.json().get('entityModelsStats')

[{'modelName': 'budget',
  'modelType': 'Entity Extractor',
  'precision': 0.4,
  'recall': 0.55,
  'fScore': 0.46},
 {'modelName': 'dst_city',
  'modelType': 'Entity Extractor',
  'precision': 0.39,
  'recall': 0.55,
  'fScore': 0.46},
 {'modelName': 'end_date',
  'modelType': 'Entity Extractor',
  'precision': 0.26,
  'recall': 0.52,
  'fScore': 0.35},
 {'modelName': 'or_city',
  'modelType': 'Entity Extractor',
  'precision': 0.43,
  'recall': 0.56,
  'fScore': 0.49},
 {'modelName': 'str_date',
  'modelType': 'Entity Extractor',
  'precision': 0.36,
  'recall': 0.55,
  'fScore': 0.43}]